In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [3]:
file = 'data/Hz31.txt'
data = pd.read_csv(file, names=['z', 'hz', 'err'], sep = " " )

# odds = [x for x in range(len(data)) if x%2 ==0]
# even = [x for x in range(len(data)) if x%2 ==1]
# new_order = np.array(even+odds)

a = np.array([25, 3, 2, 27, 14, 30, 21, 9, 8, 18, 20, 6, 22, 19, 5, 17, 4, 13, 0, 24, 29, 7, 15, 1, 12, 23, 16, 28, 10, 26, 11])
data = data.values[a]


X = data[:,0]
y = data[:,1:]

scalerx = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerx.fit(X.reshape(-1,1))
# apply transform
X = scalerx.transform(X.reshape(-1,1))


# scalery = MinMaxScaler(feature_range=(0,1))
# scalery = StandardScaler()
# scalery.fit(y)
# y = scalery.transform(y)

split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)


In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=10,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [5]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([100, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 80, 100]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([30, 50]))
HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8, 16, 32]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def autoencoder(input_z, hparams):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    ehidden = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    elast = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden)
    coded = Dense(hparams[HP_NUM_UNITS4], activation='relu')(elast)
    #decoder 
    dfirst = Dense(hparams[HP_NUM_UNITS3], activation='relu')(coded)
    dhidden = Dense(hparams[HP_NUM_UNITS2], activation='relu')(dfirst)
    dlast = Dense(hparams[HP_NUM_UNITS1], activation='relu')(dhidden)
    decoded = Dense(2, activation='relu')(dlast)
    
    return decoded

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, autoencoder(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [12]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
                      HP_NUM_UNITS1: num_units1,
                      HP_NUM_UNITS2: num_units2,
                      HP_NUM_UNITS3: num_units3,
                      HP_NUM_UNITS4: num_units4,
                    #                       HP_DROPOUT: dropout_rate,
#                       HP_OPTIMIZER: optimizer,
                      HP_BATCHSIZE: batch_size
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

--- Starting trial: run-0
{'num_units1': 100, 'num_units2': 50, 'num_units3': 30, 'num_units4': 2, 'batch_size': 4}
Epoch 1/200
6/6 [==============================] - 0s 1ms/step - loss: 6720.3979 - mean_squared_error: 6720.3979
Epoch 2/200
6/6 [==============================] - 0s 1ms/step - loss: 6710.6089 - mean_squared_error: 6710.6089
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 6694.4409 - mean_squared_error: 6694.4409
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 6663.8535 - mean_squared_error: 6663.8535
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 6608.0757 - mean_squared_error: 6608.0757
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 6498.8398 - mean_squared_error: 6498.8398
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 6304.5894 - mean_squared_error: 6304.5894
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 5984.2700 - mean_squar

Epoch 73/200
6/6 [==============================] - 0s 2ms/step - loss: 170.7454 - mean_squared_error: 170.7454
Epoch 74/200
6/6 [==============================] - 0s 1ms/step - loss: 173.3317 - mean_squared_error: 173.3317
Epoch 75/200
6/6 [==============================] - 0s 1ms/step - loss: 171.4866 - mean_squared_error: 171.4866
Epoch 76/200
6/6 [==============================] - 0s 2ms/step - loss: 171.0083 - mean_squared_error: 171.0083
Epoch 77/200
6/6 [==============================] - 0s 2ms/step - loss: 169.4895 - mean_squared_error: 169.4895
Epoch 78/200
6/6 [==============================] - 0s 1ms/step - loss: 174.1879 - mean_squared_error: 174.1879
Epoch 79/200
6/6 [==============================] - 0s 1ms/step - loss: 172.9685 - mean_squared_error: 172.9685
Epoch 80/200
6/6 [==============================] - 0s 1ms/step - loss: 176.0358 - mean_squared_error: 176.0358
Epoch 81/200
6/6 [==============================] - 0s 1ms/step - loss: 170.5266 - mean_squared_error: 1

6/6 [==============================] - 0s 1ms/step - loss: 462.6623 - mean_squared_error: 462.6623
Epoch 38/200
6/6 [==============================] - 0s 2ms/step - loss: 460.3436 - mean_squared_error: 460.3436
Epoch 39/200
6/6 [==============================] - 0s 2ms/step - loss: 456.2427 - mean_squared_error: 456.2427
Epoch 40/200
6/6 [==============================] - 0s 1ms/step - loss: 456.6569 - mean_squared_error: 456.6569
Epoch 41/200
6/6 [==============================] - 0s 1ms/step - loss: 456.5272 - mean_squared_error: 456.5272
Epoch 42/200
6/6 [==============================] - 0s 1ms/step - loss: 451.8748 - mean_squared_error: 451.8748
Epoch 43/200
6/6 [==============================] - 0s 1ms/step - loss: 456.1342 - mean_squared_error: 456.1342
Epoch 44/200
6/6 [==============================] - 0s 1ms/step - loss: 449.4635 - mean_squared_error: 449.4635
Epoch 45/200
6/6 [==============================] - 0s 2ms/step - loss: 446.8952 - mean_squared_error: 446.8952
Epoch

6/6 [==============================] - 0s 2ms/step - loss: 572.0829 - mean_squared_error: 572.0829
Epoch 13/200
6/6 [==============================] - 0s 2ms/step - loss: 554.4845 - mean_squared_error: 554.4845
Epoch 14/200
6/6 [==============================] - 0s 2ms/step - loss: 503.4935 - mean_squared_error: 503.4935
Epoch 15/200
6/6 [==============================] - 0s 2ms/step - loss: 492.5929 - mean_squared_error: 492.5929
Epoch 16/200
6/6 [==============================] - 0s 2ms/step - loss: 463.7109 - mean_squared_error: 463.7109
Epoch 17/200
6/6 [==============================] - 0s 1ms/step - loss: 395.0894 - mean_squared_error: 395.0894
Epoch 18/200
6/6 [==============================] - 0s 1ms/step - loss: 396.7633 - mean_squared_error: 396.7633
Epoch 19/200
6/6 [==============================] - 0s 1ms/step - loss: 357.3288 - mean_squared_error: 357.3288
Epoch 20/200
6/6 [==============================] - 0s 1ms/step - loss: 343.0503 - mean_squared_error: 343.0503
Epoch

6/6 [==============================] - 0s 1ms/step - loss: 168.7053 - mean_squared_error: 168.7053
Epoch 86/200
6/6 [==============================] - 0s 1ms/step - loss: 167.9034 - mean_squared_error: 167.9034
Epoch 87/200
6/6 [==============================] - 0s 1ms/step - loss: 168.2583 - mean_squared_error: 168.2583
Epoch 88/200
6/6 [==============================] - 0s 1ms/step - loss: 165.2599 - mean_squared_error: 165.2599
Epoch 89/200
6/6 [==============================] - 0s 2ms/step - loss: 165.2019 - mean_squared_error: 165.2019
Epoch 90/200
6/6 [==============================] - 0s 1ms/step - loss: 172.5402 - mean_squared_error: 172.5402
Epoch 91/200
6/6 [==============================] - 0s 1ms/step - loss: 165.9337 - mean_squared_error: 165.9337
Epoch 92/200
6/6 [==============================] - 0s 1ms/step - loss: 167.6420 - mean_squared_error: 167.6420
Epoch 93/200
6/6 [==============================] - 0s 1ms/step - loss: 167.9263 - mean_squared_error: 167.9263
Epoch

...

Epoch 84/200
1/1 [==============================] - 0s 6ms/step - loss: 6459.4844 - mean_squared_error: 6459.4844
Epoch 85/200
1/1 [==============================] - 0s 6ms/step - loss: 6459.3247 - mean_squared_error: 6459.3247
Epoch 86/200
1/1 [==============================] - 0s 4ms/step - loss: 6459.1562 - mean_squared_error: 6459.1562
Epoch 87/200
1/1 [==============================] - 0s 6ms/step - loss: 6458.9624 - mean_squared_error: 6458.9624
Epoch 88/200
1/1 [==============================] - 0s 5ms/step - loss: 6458.7407 - mean_squared_error: 6458.7407
Epoch 89/200
1/1 [==============================] - 0s 7ms/step - loss: 6458.5015 - mean_squared_error: 6458.5015
Epoch 90/200
1/1 [==============================] - 0s 6ms/step - loss: 6458.2593 - mean_squared_error: 6458.2593
Epoch 91/200
1/1 [==============================] - 0s 6ms/step - loss: 6458.0273 - mean_squared_error: 6458.0273
Epoch 92/200
1/1 [==============================] - 0s 7ms/step - loss: 6457.8164 - mean

1/1 [==============================] - 0s 5ms/step - loss: 6449.3071 - mean_squared_error: 6449.3071
Epoch 156/200
1/1 [==============================] - 0s 7ms/step - loss: 6449.1992 - mean_squared_error: 6449.1992
Epoch 157/200
1/1 [==============================] - 0s 6ms/step - loss: 6449.0977 - mean_squared_error: 6449.0977
Epoch 158/200
1/1 [==============================] - 0s 6ms/step - loss: 6449.0020 - mean_squared_error: 6449.0020
Epoch 159/200
1/1 [==============================] - 0s 6ms/step - loss: 6448.9062 - mean_squared_error: 6448.9062
Epoch 160/200
1/1 [==============================] - 0s 7ms/step - loss: 6448.8032 - mean_squared_error: 6448.8032
Epoch 161/200
1/1 [==============================] - 0s 5ms/step - loss: 6448.6953 - mean_squared_error: 6448.6953
Epoch 162/200
1/1 [==============================] - 0s 5ms/step - loss: 6448.5845 - mean_squared_error: 6448.5845
Epoch 163/200
1/1 [==============================] - 0s 5ms/step - loss: 6448.4712 - mean_squa

1/1 [==============================] - 0s 1ms/step - loss: 6413.3618 - mean_squared_error: 6413.3618
--- Starting trial: run-143
{'num_units1': 200, 'num_units2': 100, 'num_units3': 50, 'num_units4': 10, 'batch_size': 32}
Epoch 1/200
1/1 [==============================] - 0s 4ms/step - loss: 6723.4702 - mean_squared_error: 6723.4702
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 6721.8618 - mean_squared_error: 6721.8618
Epoch 3/200
1/1 [==============================] - 0s 6ms/step - loss: 6718.9688 - mean_squared_error: 6718.9688
Epoch 4/200
1/1 [==============================] - 0s 4ms/step - loss: 6715.5571 - mean_squared_error: 6715.5571
Epoch 5/200
1/1 [==============================] - 0s 5ms/step - loss: 6711.3608 - mean_squared_error: 6711.3608
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - loss: 6706.1914 - mean_squared_error: 6706.1914
Epoch 7/200
1/1 [==============================] - 0s 5ms/step - loss: 6699.7427 - mean_squared_erro

1/1 [==============================] - 0s 7ms/step - loss: 649.5091 - mean_squared_error: 649.5091
Epoch 72/200
1/1 [==============================] - 0s 6ms/step - loss: 628.7498 - mean_squared_error: 628.7498
Epoch 73/200
1/1 [==============================] - 0s 6ms/step - loss: 611.5063 - mean_squared_error: 611.5063
Epoch 74/200
1/1 [==============================] - 0s 6ms/step - loss: 599.5311 - mean_squared_error: 599.5311
Epoch 75/200
1/1 [==============================] - 0s 5ms/step - loss: 592.7871 - mean_squared_error: 592.7871
Epoch 76/200
1/1 [==============================] - 0s 6ms/step - loss: 589.9778 - mean_squared_error: 589.9778
Epoch 77/200
1/1 [==============================] - 0s 6ms/step - loss: 589.1801 - mean_squared_error: 589.1801
Epoch 78/200
1/1 [==============================] - 0s 5ms/step - loss: 588.4740 - mean_squared_error: 588.4740
Epoch 79/200
1/1 [==============================] - 0s 5ms/step - loss: 586.4042 - mean_squared_error: 586.4042
Epoch

1/1 [==============================] - 0s 7ms/step - loss: 446.7697 - mean_squared_error: 446.7697
Epoch 144/200
1/1 [==============================] - 0s 6ms/step - loss: 446.3170 - mean_squared_error: 446.3170
Epoch 145/200
1/1 [==============================] - 0s 9ms/step - loss: 445.8967 - mean_squared_error: 445.8967
Epoch 146/200
1/1 [==============================] - 0s 6ms/step - loss: 445.4963 - mean_squared_error: 445.4963
Epoch 147/200
1/1 [==============================] - 0s 8ms/step - loss: 445.1145 - mean_squared_error: 445.1145
Epoch 148/200
1/1 [==============================] - 0s 7ms/step - loss: 444.7486 - mean_squared_error: 444.7486
Epoch 149/200
1/1 [==============================] - 0s 7ms/step - loss: 444.3941 - mean_squared_error: 444.3941
Epoch 150/200
1/1 [==============================] - 0s 6ms/step - loss: 444.0496 - mean_squared_error: 444.0496
Epoch 151/200
1/1 [==============================] - 0s 8ms/step - loss: 443.7159 - mean_squared_error: 443.71

In [13]:
!kill 7893
%tensorboard --logdir logs/hparam_tuning

/bin/sh: 1: kill: No such process



Reusing TensorBoard on port 6006 (pid 30922), started 0:12:55 ago. (Use '!kill 30922' to kill it.)